# Gender Biased Document Classification

## Splitting Data into Train/Validation/Test Sets

In [1]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

Load and prepare the data for classification:

In [48]:
# df = pd.read_csv("annot_data/aggregated_with_eadid_descid_desc_cols.csv", index_col=0)
# df.head()

,file,offsets,text,label,category,eadid,description,field,id,desc_id
9,AA5_00100.ann,"(1032, 1043)",James Whyte,Masculine,Person-Name,AA5,Biographical / Historical:\nProfessor James Ai...,Biographical / Historical,0,0
16,AA5_00100.ann,"(1129, 1177)",chair of practical theology and Christian ethics,Occupation,Contextual,AA5,Biographical / Historical:\nProfessor James Ai...,Biographical / Historical,1,0
4,AA5_00100.ann,"(1217, 1219)",he,Gendered-Pronoun,Linguistic,AA5,Biographical / Historical:\nProfessor James Ai...,Biographical / Historical,2,0
5,AA5_00100.ann,"(1241, 1244)",His,Gendered-Pronoun,Linguistic,AA5,Biographical / Historical:\nProfessor James Ai...,Biographical / Historical,3,0
6,AA5_00100.ann,"(1315, 1317)",he,Gendered-Pronoun,Linguistic,AA5,Biographical / Historical:\nProfessor James Ai...,Biographical / Historical,4,0


In [49]:
# df[["field2", "description"]] = df["description"].str.split(":", n=1, expand=True)
# df.drop("field2", axis=1, inplace=True)
# df.head()
# df.drop(["file", "offsets", "text", "category", "eadid"], axis=1, inplace=True)
# descs = list(df.description)
# descs = [d.strip() for d in descs]
# df.description = descs
# df.to_csv("data/aggregated_data_for_classifier.csv")

In [2]:
df = pd.read_csv("data/aggregated_data_for_classifier.csv", index_col=0)
df.head()

,label,description,field,id,desc_id
9,Masculine,Professor James Aitken White was a leading Sco...,Biographical / Historical,0,0
16,Occupation,Professor James Aitken White was a leading Sco...,Biographical / Historical,1,0
4,Gendered-Pronoun,Professor James Aitken White was a leading Sco...,Biographical / Historical,2,0
5,Gendered-Pronoun,Professor James Aitken White was a leading Sco...,Biographical / Historical,3,0
6,Gendered-Pronoun,Professor James Aitken White was a leading Sco...,Biographical / Historical,4,0


In [3]:
df.label.unique()

array(['Masculine', 'Occupation', 'Gendered-Pronoun', 'Stereotype',
       'Unknown', 'Feminine', 'Generalization', 'Omission',
       'Gendered-Role'], dtype=object)

The DataFrame above has a row for every label, so for descriptions with multiple labels, there are multiple rows of data.

In [4]:
df.description.isnull().values.any()

False

For every description, get the labels (non-repeating) they were annotated with:

In [26]:
df_grouped = df.groupby(["description", "field", "desc_id"]).agg(
    {"label": lambda label_name: set(label_name)}  #",".join(label_name)}
    ).reset_index()
df_grouped.sort_values(by="desc_id", inplace=True)
df_grouped.head()

,description,field,desc_id,label
8054,Professor James Aitken White was a leading Sco...,Biographical / Historical,0,"{Masculine, Gendered-Pronoun, Occupation, Ster..."
5560,Papers of The Very Rev Prof James Whyte (1920-...,Title,1,"{Masculine, Stereotype, Unknown}"
8325,Rev Thomas Allan was born on 16 August 1916 in...,Biographical / Historical,2,"{Generalization, Omission, Masculine, Feminine..."
5550,Papers of Rev Tom Allan (1916-1965),Title,3,"{Masculine, Unknown}"
633,Alec Cheyne was born on 1 June 1924 in Errol i...,Biographical / Historical,4,"{Generalization, Omission, Masculine, Feminine..."


Add a column for the EADIDs (collection, or fonds, identifier) of the descriptions:

In [22]:
descs = pd.read_csv("data/descriptions.csv", index_col=0)
eadids = descs.drop(columns=["description","field"])
# descs.head()
eadids.head()

,eadid,desc_id
9,AA5,0
17,AA5,1
39,AA6,2
47,AA6,3
70,AA7,4


In [27]:
key = "desc_id"
df_joined = df_grouped.join(eadids.set_index(key), on=key)
df_joined.head()

,description,field,desc_id,label,eadid
8054,Professor James Aitken White was a leading Sco...,Biographical / Historical,0,"{Masculine, Gendered-Pronoun, Occupation, Ster...",AA5
5560,Papers of The Very Rev Prof James Whyte (1920-...,Title,1,"{Masculine, Stereotype, Unknown}",AA5
8325,Rev Thomas Allan was born on 16 August 1916 in...,Biographical / Historical,2,"{Generalization, Omission, Masculine, Feminine...",AA6
5550,Papers of Rev Tom Allan (1916-1965),Title,3,"{Masculine, Unknown}",AA6
633,Alec Cheyne was born on 1 June 1924 in Errol i...,Biographical / Historical,4,"{Generalization, Omission, Masculine, Feminine...",AA7


Write the resulting DataFrame to a file:

In [29]:
df_joined.to_csv("data/desc_field_descid_label_eadid.csv")

### OPTION 1:

In [5]:
# ---------------------------------
# Uncomment if need to reload data
# ---------------------------------
df_joined = pd.read_csv("data/desc_field_descid_label_eadid.csv", index_col=0)

Create subsets of the DataFrame for each field type:

In [6]:
field_names = df_joined.field.unique()
print(field_names)

['Biographical / Historical' 'Title' 'Scope and Contents'
 'Processing Information']


In [7]:
df_bh = df_joined.loc[df_joined.field == field_names[0]]
df_t = df_joined.loc[df_joined.field == field_names[1]]
df_sc = df_joined.loc[df_joined.field == field_names[2]]
df_pi = df_joined.loc[df_joined.field == field_names[3]]
df_pi.head()

,description,field,desc_id,label,eadid
833,Archivist's NoteNone Grant Buttars 28 April 2003,Processing Information,393,"{'Unknown', 'Occupation'}",BAI
821,Archivist's NoteNone Grant Buttars 22 May 2003,Processing Information,394,"{'Unknown', 'Occupation'}",BAI
810,Archivist's NoteNone Grant Buttars 15 May 2003,Processing Information,395,"{'Unknown', 'Occupation'}",BAI
790,Archivist's NoteNone Grant Buttars 03 June 2003,Processing Information,396,"{'Unknown', 'Occupation'}",BAI
811,Archivist's NoteNone Grant Buttars 16 April 2003,Processing Information,397,"{'Unknown', 'Occupation'}",BAI


Shuffle each DataFrame and then add a column to each that assigns every row to a subset.  For each DataFrame, assign: 
* 60% of the rows to `training`
* 20% of the rows to `validation`
* 20% of the rows to `test`

In [10]:
# INPUT: DataFrame, fraction of DF to shuffle, and random_state of shuffle
#        Note 1 - fraction defaults to 1 to shuffle the entire DataFrame; provide a value <1 to return that fraction of the DataFrame shuffled
#        Note 2 -random_state_value defaults to 7 for reproducibility
# OUTPUT: DataFrame with its rows shuffled
def shuffleDataFrame(df, fraction=1, random_state_value=7):
    return df.sample(frac=fraction, random_state=random_state_value)

In [11]:
# Shuffle the DataFrames for each metadata field type
df_bh_shuffled = shuffleDataFrame(df_bh)
df_t_shuffled = shuffleDataFrame(df_t)
df_sc_shuffled = shuffleDataFrame(df_sc)
df_pi_shuffled = shuffleDataFrame(df_pi)
df_t_shuffled.head()

,description,field,desc_id,label,eadid
4766,List of Mamaine Koestler's estate ::,Title,6770,{'Unknown'},Coll-146
5288,Notes titled On Horner's Method of Solving Equ...,Title,2606,"{'Omission', 'Unknown', 'Generalization'}",Coll-1310
2896,File relating to foreign rights and earnings f...,Title,3425,"{'Masculine', 'Unknown'}",Coll-1320
3723,"Letter :: Baker, Penny",Title,7814,{'Unknown'},Coll-146
2646,Dowager 3d and Molly Millicent,Title,4362,{'Unknown'},Coll-1434


In [31]:
def getTrainValTestSizes(df):
    indeces = list(df.index)
    
    train = indeces[ : int(df.shape[0]*0.6) ]
    validate = indeces[ int(df.shape[0]*0.6) : (int(df.shape[0]*0.6) + round(df.shape[0]*0.2)) ]
    test = indeces[ (int(df.shape[0]*0.6) + round(df.shape[0]*0.2)) : ]

    return len(train), len(validate), len(test)

In [32]:
# Get the indeces of rows to assign to train, dev, and test
train_bh, validate_bh, test_bh = getTrainValTestSizes(df_bh_shuffled)
assert train_bh+validate_bh+test_bh == df_bh_shuffled.shape[0]

train_t, validate_t, test_t = getTrainValTestSizes(df_t_shuffled)
assert train_t+validate_t+test_t == df_t_shuffled.shape[0]

train_sc, validate_sc, test_sc = getTrainValTestSizes(df_sc_shuffled)
assert train_sc+validate_sc+test_sc == df_sc_shuffled.shape[0]

train_pi, validate_pi, test_pi = getTrainValTestSizes(df_pi_shuffled)
assert train_pi+validate_pi+test_pi == df_pi_shuffled.shape[0]

In [38]:
# Add a column to the input DataFrame that assigns each row to train, dev, and test
# using the three input sizes
def assignSubsets(df, train_size, validate_size, test_size):
    subset_col = ["train"]*train_size + ["dev"]*validate_size + ["test"]*test_size
    df.insert(len(df.columns)-1, "subset", subset_col)
    return df

In [41]:
df_bh = assignSubsets(df_bh_shuffled, train_bh, validate_bh, test_bh)
# df_bh.head()
df_t = assignSubsets(df_t_shuffled, train_t, validate_t, test_t)
df_sc = assignSubsets(df_sc_shuffled, train_sc, validate_sc, test_sc)
df_pi = assignSubsets(df_pi_shuffled, train_pi, validate_pi, test_pi)

Concatenate the rows assigned to each subset to create one DataFrame each for training, validation, and testing: 

In [42]:
def concatBySubset(df_list, subset):
    df_all = pd.DataFrame()
    for df in df_list:
        df_subset = df.loc[df["subset"] == subset]
        df_all = pd.concat([df_all, df_subset], axis=0)
    return df_all

In [48]:
dfs = [df_bh, df_t, df_sc, df_pi]

train = concatBySubset(dfs, "train")
assert train.subset.unique()[0] == "train"

validate = concatBySubset(dfs, "dev")
assert validate.subset.unique()[0] == "dev"

test = concatBySubset(dfs, "test")
assert test.subset.unique()[0] == "test"


In [49]:
splits = [train, validate, test]
for split in splits:
    print(split.shape[0])


7046
2350
2352


In [52]:
print(7046/(7045+2350+2352))
print(2350/(7045+2350+2352))
print(2352/(7045+2350+2352))



0.599812718140802
0.20005107687069038
0.200221333106325


Great!  Now that we've split the data up into three subsets of 60%, 20% and 20%, we can write the corresponding labels and descriptions to files for creating classification models.

#### Write the data to files
The files will separate labels by `\n` (a newline) and descriptions by `\n|\n` (a pipe character surrounded by newlines)

In [65]:
def writeFiles(df, filename, directory = "clf_data/", docs=True):
    if docs:
        rows = list(df.description)
    else:
        rows = list(df.label)
    for i,text in enumerate(rows):
        f = open(filename, "a")
        if docs:
            f.write(text+"\n|\n")
        else:
            f.write(text+"\n")
    f.close()
    print("Your file has been written!")

In [66]:
train_docs = directory+"train_docs.txt"
writeFiles(train, train_docs)

train_labels = directory+"train_labels.txt"
writeFiles(train, train_labels, False)

Your file has been written!
Your file has been written!


In [67]:
validate_docs = directory+"validate_docs.txt"
writeFiles(validate, validate_docs)

validate_labels = directory+"validate_labels.txt"
writeFiles(validate, validate_labels, False)

Your file has been written!
Your file has been written!


In [68]:
test_docs = directory+"blindtest_docs.txt"
writeFiles(test, test_docs)

test_labels = directory+"blindtest_labels.txt"
writeFiles(test, test_labels, False)

Your file has been written!
Your file has been written!


***

### OPTION 2: 

array for documents and array for labels

Format data as a numpy array of strings, where the input data are each one description as a string, and the target data are strings, one for each description, with comma-separated label names begins with the description and ending with comma-separated labels.  For example:

* Input (document): `"Professor James Aitken White was a leading Scottish Theologian."`
* Target (document's labels): `"Stereotype,Masculine,Gendered-Pronoun,Occupation"`

In [19]:
docs = list(dl_dict.keys())   # input documents (descriptions)
labels = list(dl_dict.values()) # target labels

Shuffle and split data into train and test sets: 

*Note: least populated class has too few groups to stratify*

In [20]:
X_train, X_test, y_train, y_test = train_test_split(docs, labels, test_size=0.2, random_state=0, shuffle=True)

Shuffle and split the train dataset into train and validation sets:

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0, shuffle=True)

#### Write the data to files

In [25]:
# directory: clf_data for option 1 / clf_data2 for option2
train_file = "clf_data2/train_docs.txt"
for i,doc in enumerate(X_train):
    f_train = open(train_file, "a")
    f_train.write(doc+"\n|\n")
f_train.close()

In [26]:
train_file = "clf_data2/train_labels.txt"
for i,labels in enumerate(y_train):
    f_train = open(train_file, "a")
    f_train.write(labels+"\n")
f_train.close()

In [27]:
val_file = "clf_data2/validation_docs.txt"
for i,doc in enumerate(X_val):
    f_val = open(val_file, "a")
    f_val.write(doc+"\n|\n")
f_val.close()

In [28]:
val_file = "clf_data2/validation_labels.txt"
for i,labels in enumerate(y_val):
    f_val = open(val_file, "a")
    f_val.write(labels+"\n")
f_val.close()

In [29]:
test_file = "clf_data2/blindtest_doc.txt"
for i,doc in enumerate(X_test):
    f_test = open(test_file, "a")
    f_test.write(doc+"\n|\n")
f_test.close()

In [30]:
test_file = "clf_data2/blindtest_labels.txt"
for i,labels in enumerate(y_test):
    f_test = open(test_file, "a")
    f_test.write(labels+"\n")
f_test.close()

In [11]:
# vectorizer = TfidfVectorizer(lowercase=False)               # defaults input="content", encoding="utf8"

In [12]:
# X = vectorizer.fit_transform(docs)   # archival metadata descriptions (one description = one document)

In [14]:
# y = vectorizer.fit_transform(labels)         # corresponding gender biased language labels for each description
# y = np.array(labels)

In [21]:
# sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=0) # set random_state for reproducibility
# fold = 1
# for train,test in sss.split(X, y):
#     train_file = "clf_data/fold"+str(fold)+"/train.txt"
#     f_train = open(train_file, "w")
#     f_train.write(train)
#     f_train.close()
    
#     test_file = "clf_data/fold"+str(fold)+"test.txt"
#     f_test = open(test_file, "w")
#     f_test.write(test)
#     f_test.close()
    
#     fold += 1

In [ ]:
# # INPUT: DataFrame of annotated descriptions with columns for description, field, and label
# # OUTPUT: 3D array with columns for the description, metadata field name, and label
# def desc_field_label_extractor(df):
#     descs = list(df.description)
#     fields = list(df.field)
#     labels = list(df.label)
#     features = np.empty(shape=(len(descs), 3), dtype=object)
#     for i,desc in enumerate(descs):    
#         features[i, 0] = desc
#         features[i, 1] = fields[i]
#         features[i, 2] = labels[i]
#     return features


# desc_field_label_transformer = FunctionTransformer(desc_field_label_extractor)